# evaluate WER of whisper with assistant

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phineas-pta/fine-tune-whisper-vi/blob/main/eval/evaluate-assistant.ipynb)

**DOESN’T WORK**: weird error about mel spectrogram

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
# !huggingface-cli login --token=███

In [ ]:
# workaround for a bug in `datasets` package
%pip uninstall -y cudf dask-cuda dask-cudf
%pip install -q cudf-cu12 --extra-index-url=https://pypi.nvidia.com
%pip install -qU 'datasets[audio]' accelerate transformers jiwer
# install then `import evaluate` throw error on kaggle

In [ ]:
from tqdm import tqdm
import torch
from transformers import pipeline, AutoModelForSpeechSeq2Seq
import datasets as hugDS
import jiwer
# DO NOT USE `evaluate.evaluator`: buggy, cannot set language resulting very bad WER

JIWER_TRANS = jiwer.Compose([  # DO NOT use `jiwer.RemoveEmptyStrings` it can cause rows count mismatch
	jiwer.ToLowerCase(),
	jiwer.RemoveKaldiNonWords(),
	jiwer.RemoveMultipleSpaces(),
	jiwer.Strip(),
	jiwer.RemovePunctuation(),
	jiwer.ReduceToListOfListOfWords(),
])

In [ ]:
SAMPLING_RATE = 16_000
def load_my_data(**kwargs):
	return hugDS.load_dataset(**kwargs, split="test", trust_remote_code=True, streaming=True).cast_column("audio", hugDS.Audio(sampling_rate=SAMPLING_RATE))

MY_DATA = hugDS.IterableDatasetDict()
MY_DATA["commonvoice"] = load_my_data(path="mozilla-foundation/common_voice_16_1", name="vi",  ).select_columns(["audio", "sentence"])
# MY_DATA["fleurs"] # disable FLEURS because error with tensor size mismatch when batching, see bottom for non-batched inference
MY_DATA["vivos"]       = load_my_data(path="vivos"                                             ).select_columns(["audio", "sentence"])
MY_DATA["bud500"]      = load_my_data(path="linhtran92/viet_bud500"                            ).rename_column("transcription", "sentence")
MY_DATA["lsvsc"]       = load_my_data(path="doof-ferb/LSVSC"                                   ).select_columns(["audio", "transcription"]).rename_column("transcription", "sentence")

ROWS_COUNT = {
	"commonvoice": 1326,
	"fleurs":       857,
	"vivos":        760,
	"bud500":      7500,
	"lsvsc":       5683,
}

In [ ]:
DTYPE = torch.float16
DEV = "cuda:0"
PIPE = pipeline(task="automatic-speech-recognition", model="vinai/PhoWhisper-large", device=DEV, torch_dtype=DTYPE)
ASSISTANT = AutoModelForSpeechSeq2Seq.from_pretrained("doof-ferb/whisper-tiny-vi", torch_dtype=DTYPE).to(DEV)
PIPE_KWARGS = {"language": "vi", "task": "transcribe", "assistant_model": ASSISTANT}

In [ ]:
# workaround because KeyDataset(MY_DATA[split], "audio") raise error with streaming datasets
def data(batch):
	for row in batch:
		yield row["audio"]


@torch.inference_mode()
def predict(split: str) -> tuple[list[str]]:
	batch = MY_DATA[split]
	y_pred = [out["text"] for out in tqdm(PIPE(data(batch), generate_kwargs=PIPE_KWARGS), total=ROWS_COUNT[split], unit="samples", desc=f"{split=}")]
	y_true = [row["sentence"] for row in batch]
	assert len(y_pred) == len(y_true)
	return y_true, y_pred


for split in MY_DATA.keys():
	y_true, y_pred = predict(split)
	torch.cuda.empty_cache()  # forced clean
	wer = 100 * jiwer.wer(
		reference=y_true,
		hypothesis=y_pred,
		reference_transform=JIWER_TRANS,
		hypothesis_transform=JIWER_TRANS,
	)
	if 0 < wer < 100:
		print(f"WER on {split} = {wer:.1f}%", end="\n\n")
	else:
		print("something wrong, check 5 first & last transcription:")
		print(y_true[:5], y_true[-5:])
		print(y_pred[:5], y_pred[-5:])


In [ ]:
data_fleurs = load_my_data(path="google/fleurs", name="vi_vn", streaming=False).select_columns(["audio", "transcription"])

@torch.autocast(device_type="cuda")  # required by PEFT
@torch.inference_mode()
def predict_fleurs(batch):
	batch["pred"] = PIPE(batch["audio"], generate_kwargs=PIPE_KWARGS)["text"]
	return batch
data_fleurs = data_fleurs.map(predict_fleurs)  # progress bar included

wer = 100 * jiwer.wer(
	reference=data_fleurs["transcription"],
	hypothesis=data_fleurs["pred"],
	reference_transform=JIWER_TRANS,
	hypothesis_transform=JIWER_TRANS,
)
print(f"WER on FLEURS = {wer:.1f}%", end="\n\n")